<a href="https://colab.research.google.com/github/waqasde/AIQ2_Waqas_Ahmad_PIAIC139128/blob/main/Agentic_AI_Chatbot_Project_01_User_Chat_Interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_anthropic

In [3]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic AI Chatbot Prototype 1"

gemini_api_key = userdata.get('GEMINI_API_KEY')

In [5]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [7]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [9]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [16]:
from typing import Annotated, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, ToolMessage, SystemMessage
from pydantic import BaseModel
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool
    ask_user: bool
    long_term_memory: dict  # To personalize answers based on prior interactions

class RequestAssistance(BaseModel):
    """Escalate the conversation to an expert."""
    request: str

# Define the system message to enforce healthcare context
system_prompt = SystemMessage(
    content=(
        "You are a healthcare assistant chatbot. Only answer questions related to healthcare topics, such as "
        "medicine, diseases, symptoms, treatments, therapies, diagnoses, hospitals, or doctors. "
        "If a query is unrelated to healthcare, politely respond that you can only assist with healthcare-related queries."
    )
)

# Initialize tools
tool = TavilySearchResults(max_results=5)
tools = [tool]

# Initialize LLM with tools and personalization
llm_with_tools = llm.bind_tools(tools + [RequestAssistance])

def chatbot(state: State):
    # Inject the system prompt if it's not already included
    if not any(isinstance(msg, SystemMessage) for msg in state["messages"]):
        state["messages"].insert(0, system_prompt)

    user_message = state["messages"][-1].content if state["messages"] else ""

    # Check for personalization
    if "user_name" in state["long_term_memory"]:
        personalized_greeting = f"Hello, {state['long_term_memory']['user_name']}! "
        user_message = personalized_greeting + user_message

    # Pause and ask the user for more information if needed
    if "?" in user_message and len(user_message.split()) < 5:
        state["ask_user"] = True
        return {
            "messages": [
                AIMessage(content="Could you please provide more details about your question?")
            ],
            "ask_user": True,
        }

    # Escalate to a human assistant if the chatbot is unsure
    if "I don't know" in user_message or "uncertain" in user_message:
        state["ask_human"] = True
        return {
            "messages": [
                AIMessage(content="I'm unsure about this. Let me connect you to a human assistant for further help.")
            ],
            "ask_human": True,
        }

    response = llm_with_tools.invoke(state["messages"])
    ask_human = False
    if (
        response.tool_calls
        and response.tool_calls[0]["name"] == RequestAssistance.__name__
    ):
        ask_human = True

    # Save to long-term memory if personal information is shared
    if "my name is" in user_message.lower():
        name = user_message.split("my name is")[-1].strip()
        state["long_term_memory"]["user_name"] = name

    return {"messages": [response], "ask_human": ask_human, "ask_user": False}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=[tool]))

def create_response(response: str, ai_message: AIMessage):
    return ToolMessage(
        content=response,
        tool_call_id=ai_message.tool_calls[0]["id"],
    )

def human_node(state: State):
    return {
        "messages": [
            AIMessage(content="A human assistant has been notified and will assist you shortly.")
        ],
        "ask_human": False,
    }

def user_interaction_node(state: State):
    return {
        "messages": [
            AIMessage(content="Thanks for the clarification. Let me continue assisting you."),
        ],
        "ask_user": False,
    }

graph_builder.add_node("human", human_node)
graph_builder.add_node("user_interaction", user_interaction_node)

def select_next_node(state: State):
    if state["ask_human"]:
        return "human"
    if state["ask_user"]:
        return "user_interaction"
    return tools_condition(state)

graph_builder.add_conditional_edges(
    "chatbot",
    select_next_node,
    {"human": "human", "user_interaction": "user_interaction", "tools": "tools", END: END},
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")
graph_builder.add_edge("user_interaction", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["human", "user_interaction"],
)


In [17]:
from typing import Any

# Function to stream graph updates
def stream_graph_updates(user_input: str):
    """
    Streams updates from the chatbot graph based on user input.

    Args:
        user_input (str): The input provided by the user.
    """
    # Initialize state with default values
    state = {
        "messages": [("user", user_input)],
        "ask_human": False,
        "ask_user": False,
        "long_term_memory": {},  # Initialize long-term memory
    }

    # Configuration for graph streaming
    config = {"configurable": {"thread_id": "1"}}

    # Validate user input before processing
    if not user_input.strip():
        print("Assistant: It seems you didn't enter anything. Please ask a healthcare-related question.")
        return

    try:
        # Stream responses from the graph
        for event in graph.stream(state, config):
            for value in event.values():
                print("Assistant:", value["messages"][-1].content)
    except KeyError as e:
        print(f"KeyError: {e} - Ensure all required keys are initialized in the state.")
    except Exception as e:
        print(f"An error occurred: {e}")


def main():
    """
    Main function to run the interactive healthcare chatbot.
    """
    print("Welcome to the Healthcare Chatbot! Ask me simple health-related questions.")
    print("Type 'quit', 'exit', or 'q' to end the conversation.")

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()

            # Exit condition
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye! Take care of your health.")
                break

            # Stream graph updates with user input
            stream_graph_updates(user_input)
        except KeyboardInterrupt:
            # Graceful exit on Ctrl+C
            print("\nGoodbye! Take care of your health.")
            break
        except Exception as e:
            # Handle unexpected errors
            print(f"An unexpected error occurred: {e}")
            break


if __name__ == "__main__":
    main()


Welcome to the Healthcare Chatbot! Ask me simple health-related questions.
Type 'quit', 'exit', or 'q' to end the conversation.
You: diabetes
Assistant: Diabetes is a chronic metabolic disorder characterized by elevated levels of blood sugar (glucose).  There are several types, including type 1, type 2, and gestational diabetes.  Each has different causes and management strategies.  To provide you with more specific information, could you tell me what you'd like to know about diabetes?  For example, are you interested in the symptoms, causes, treatments, or management of a particular type of diabetes?

You: what is treatments
Assistant: Diabetes treatments vary depending on the type of diabetes and the individual's specific needs.  There's no single "cure," but treatments aim to manage blood sugar levels and prevent complications.

**Type 1 Diabetes:** This type requires lifelong insulin therapy, typically through injections or an insulin pump.  Careful monitoring of blood sugar levels